# CIFAR Table 1

TJ Kim

1.17.22

#### Summary:
- Make a table for Benign transferability and inter-boundary distance for following models
- Local benign, fedavg benign, fedEM benign

Updated: 10.7.22
Manipulate sub-groups to obtain standard deviation

In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

#### Local Benign

In [1]:
# Load blackbox model
setting = 'FedAvg'

if setting == 'FedEM':
    n = 3
else:
    n = 1

adv_mode = True

# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = setting
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= n
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/dummy/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=40)

NameError: name 'Args' is not defined

Generate Dataset to be used throughout all analysis.

In [2]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
data_y = torch.stack(data_y)

NameError: name 'clients' is not defined

Load local model.

In [31]:
# setting = 'FedEM'
num_models = 40

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

if setting == 'local':
    
    if adv_mode:
        args_.save_path ='weights/final/cifar/fig3/local_defend'
    else: 
        args_.save_path = 'weights/final/cifar/fig1/Local_Benign/'
    aggregator.load_state(args_.save_path)
    
    model_weights = []
    
    if adv_mode:
        weights = np.load('weights/final/cifar/fig3/local_defend/train_client_weights.npy')
    else:
        weights = np.load("weights/final/cifar/fig1/Local_Benign/train_client_weights.npy")
    
    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for i in range(num_models):
        new_model = copy.deepcopy(aggregator.clients[i].learners_ensemble.learners[0].model)
        new_model.eval()
        models_test += [new_model]

elif setting == 'FedAvg':
    
    if adv_mode:
        args_.save_path = 'weights/final/cifar/fig3/fed_avg_defend'
    else:
        args_.save_path = 'weights/final/cifar/fig1/FedAvg_Benign/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

    if adv_mode:
        weights = np.load('weights/final/cifar/fig3/fed_avg_defend/train_client_weights.npy')
    else: 
        weights = np.load("weights/final/cifar/fig1/FedAvg_Benign/train_client_weights.npy")
    
    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0[0]*weights_h[0][key] 
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

elif setting == 'FedEM':
    
    if adv_mode: 
        args_.save_path = 'weights/final/cifar/fig3/fedem_defend/'
    else: 
        args_.save_path = 'weights/cifar/21_12_30_feddef_n40_linf0_5_G0_0/'

    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

    if adv_mode:
        weights = np.load("weights/final/cifar/fig3/fedem_defend/train_client_weights.npy")
    else:
        weights = np.load("weights/cifar/21_12_30_feddef_n40_linf0_5_G0_0/train_client_weights.npy")

    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]


    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0,w1,w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

Set Up Transfer Attack Scenario

In [37]:
# divide models into 5 sub-groups and run 
num_groups = 5
vic_dic ={}
sub_user = 8

for i in range(num_groups):
    vic_dic[i] = range(i*sub_user, (i+1)*sub_user)

In [33]:
upper_logs = []

for j in range(num_groups):
    logs_adv = []

    for i in range(num_models):
        adv_dict = {}
        adv_dict['orig_acc_transfers'] = None
        adv_dict['orig_similarities'] = None
        adv_dict['adv_acc_transfers'] = None
        adv_dict['adv_similarities_target'] = None
        adv_dict['adv_similarities_untarget'] = None
        adv_dict['adv_target'] = None
        adv_dict['adv_miss'] = None
        adv_dict['metric_alignment'] = None
        adv_dict['ib_distance_legit'] = None
        adv_dict['ib_distance_adv'] = None

        logs_adv += [adv_dict]
    upper_logs += [logs_adv]

In [34]:
rec_dict = {}

for i in range(num_groups):
    # Run Measurements for both targetted and untargeted analysis
    victim_idxs = vic_dic[i]
    custom_batch_size = 500


    for adv_idx in victim_idxs:
        print("\t Adv idx:", adv_idx)

        dataloader = load_client_data(clients = clients, c_id = adv_idx, mode = 'test') # or test/train

        batch_size = min(custom_batch_size, dataloader.y_data.shape[0])

        t1 = Transferer(models_list=models_test, dataloader=dataloader)
        t1.generate_victims(victim_idxs)

        # Perform Attacks
        t1.atk_params = PGD_Params()
        t1.atk_params.set_params(batch_size=batch_size, iteration = 20,
                       target = 3, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                       step_size = 0.05, step_norm = "inf", eps = 4.5, eps_norm = 2)



        t1.generate_advNN(adv_idx)
        t1.generate_xadv(atk_type = "pgd")
        t1.send_to_victims(victim_idxs)

        # Log Performance
        upper_logs[i][adv_idx]['orig_acc_transfers'] = copy.deepcopy(t1.orig_acc_transfers)
        upper_logs[i][adv_idx]['orig_similarities'] = copy.deepcopy(t1.orig_similarities)
        upper_logs[i][adv_idx]['adv_acc_transfers'] = copy.deepcopy(t1.adv_acc_transfers)
        upper_logs[i][adv_idx]['adv_similarities_target'] = copy.deepcopy(t1.adv_similarities)        
        upper_logs[i][adv_idx]['adv_target'] = copy.deepcopy(t1.adv_target_hit)

        # Miss attack
        t1.atk_params.set_params(batch_size=batch_size, iteration = 20,
                       target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                       step_size = 0.05, step_norm = "inf", eps = 4.5, eps_norm = 2)
        t1.generate_xadv(atk_type = "pgd")
        t1.send_to_victims(victim_idxs)
        upper_logs[i][adv_idx]['adv_miss'] = copy.deepcopy(t1.adv_acc_transfers)
        upper_logs[i][adv_idx]['adv_similarities_untarget'] = copy.deepcopy(t1.adv_similarities)

	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 8
	 Adv idx: 9
	 Adv idx: 10
	 Adv idx: 11
	 Adv idx: 12
	 Adv idx: 13
	 Adv idx: 14
	 Adv idx: 15
	 Adv idx: 16
	 Adv idx: 17
	 Adv idx: 18
	 Adv idx: 19
	 Adv idx: 20
	 Adv idx: 21
	 Adv idx: 22
	 Adv idx: 23
	 Adv idx: 24
	 Adv idx: 25
	 Adv idx: 26
	 Adv idx: 27
	 Adv idx: 28
	 Adv idx: 29
	 Adv idx: 30
	 Adv idx: 31
	 Adv idx: 32
	 Adv idx: 33
	 Adv idx: 34
	 Adv idx: 35
	 Adv idx: 36
	 Adv idx: 37
	 Adv idx: 38
	 Adv idx: 39


Print Relevant Information

In [35]:
metrics = ['orig_acc_transfers','orig_similarities','adv_acc_transfers','adv_similarities_target',
           'adv_similarities_untarget','adv_target','adv_miss']

adv_targets = []
adv_misses = []
orig_accs = []

for i in range(num_groups):
    victim_idxs = vic_dic[i]
    
    orig_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
    orig_sim = np.zeros([len(victim_idxs),len(victim_idxs)]) 
    adv_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
    adv_sim_target = np.zeros([len(victim_idxs),len(victim_idxs)]) 
    adv_sim_untarget = np.zeros([len(victim_idxs),len(victim_idxs)]) 
    adv_target = np.zeros([len(victim_idxs),len(victim_idxs)])
    adv_miss = np.zeros([len(victim_idxs),len(victim_idxs)]) 

    for adv_idx in range(len(victim_idxs)):
        for victim in range(len(victim_idxs)):
            orig_acc[adv_idx,victim] = upper_logs[i][victim_idxs[adv_idx]][metrics[0]][victim_idxs[victim]].data.tolist()
            orig_sim[adv_idx,victim] = upper_logs[i][victim_idxs[adv_idx]][metrics[1]][victim_idxs[victim]].data.tolist()
            adv_acc[adv_idx,victim] = upper_logs[i][victim_idxs[adv_idx]][metrics[2]][victim_idxs[victim]].data.tolist()
            adv_sim_target[adv_idx,victim] = upper_logs[i][victim_idxs[adv_idx]][metrics[3]][victim_idxs[victim]].data.tolist()
            adv_sim_untarget[adv_idx,victim] = upper_logs[i][victim_idxs[adv_idx]][metrics[4]][victim_idxs[victim]].data.tolist()
            adv_target[adv_idx,victim] = upper_logs[i][victim_idxs[adv_idx]][metrics[5]][victim_idxs[victim]].data.tolist()
            adv_miss[adv_idx,victim] = upper_logs[i][victim_idxs[adv_idx]][metrics[6]][victim_idxs[victim]].data.tolist()
            
    adv_targets += [avg_nondiag(adv_target)]
    adv_misses += [avg_nondiag(adv_miss)]
    orig_accs += [np.mean(np.diagonal(orig_acc))]

In [36]:
print('adv_target --', 'mean:', np.mean(adv_targets), 'sd:', np.std(adv_target))
print('adv_miss --', 'mean:', np.mean(adv_misses), 'sd:', np.std(adv_miss))
print('orig_acc --', 'mean:', np.mean(orig_accs), 'sd:', np.std(orig_acc))

adv_target -- mean: 0.13329238557310488 sd: 0.14267922015366863
adv_miss -- mean: 0.4826734611334709 sd: 0.11925628370996559
orig_acc -- mean: 0.6614128015935421 sd: 0.07135116505130504


Set Up Interboundary Measure Scenario

In [38]:
num_trials = 5
batch_size = 50
adv_idxs = [0,8,16,24,32]
group_list = [0,1]

# Create dataloader from validation dataset that allows for diverse batch size
dataloader = Custom_Dataloader(data_x, data_y)
num_model2 = 8

dmls = []
dma = []
dmae = []

for i in group_list:
    victim_idxs = vic_dic[i]
    adv_idx = adv_idxs[i]

    dists_measure_legit = np.zeros([num_trials, num_model2-1])
    dists_measure_adv = np.zeros([num_trials, num_model2-1])
    dists_measure_adv_ensemble = np.zeros([num_trials, num_model2-1])


#     for j in range(num_model2):
    for i in range(num_trials):
        print("num_trial:", i)
        t1 = Boundary_Transferer(models_list=models_test[victim_idxs[0]:(victim_idxs[-1]+1)], dataloader=dataloader)
        t1.base_nn_idx = 0
        t1.victim_idx = list(range(1, num_model2))

        t1.atk_params = PGD_Params()
        t1.atk_params.set_params(batch_size=500, iteration = 30,
                       target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                       step_size = 0.05, step_norm = "inf", eps = 3, eps_norm = 2)
        t1.set_adv_NN(t1.base_nn_idx)

        base_ep_legit, victim_eps_legit = t1.legitimate_direction(batch_size=batch_size, ep_granularity = 0.5, 
                                                                  rep_padding = 200, new_point = True,print_res = False)

        base_ep_adv, victim_eps_adv = t1.adversarial_direction(ep_granularity = 0.5, 
                                                                  rep_padding = 200, new_point = False,print_res = False)

        idx = 0
        for key, value in victim_eps_legit.items():
            dists_measure_legit[i,idx] = np.abs(base_ep_legit-value)
            idx+=1

        idx = 0
        for key, value in victim_eps_adv.items():
            dists_measure_adv[i,idx] = np.abs(base_ep_adv - value)
            idx+=1
                
    dmls += [dists_measure_legit]
    dma += [dists_measure_adv]
    

num_trial: 0
num_trial: 1
num_trial: 2
num_trial: 3
num_trial: 4
num_trial: 0
num_trial: 1
num_trial: 2
num_trial: 3
num_trial: 4


In [39]:
np.mean(np.average(dmls,axis=1))

8.814285714285713

In [40]:
np.std(np.average(dmls,axis=1))

3.878117574077318

In [41]:
np.mean(np.average(dma,axis=1))

32.65714285714285

In [42]:
np.std(np.average(dma,axis=1))

18.606708248283326

#### Logs

In [18]:
# Global Validation Dataset

# Local - Benign
local_adv_miss = 0.37
local_adv_target = 0.07
local_orig_acc = 0.41
local_ibdist_legit = 39.52875
local_ibdist_adv = 49.74

# Fedavg - Benign
fedavg_adv_miss = 0.0
fedavg_adv_target = 0.86 
fedavg_orig_acc = 0.96
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - Benign
fedem_adv_miss = 0.12
fedem_adv_target = 0.43
fedem_orig_acc = 0.79
fedem_ibdist_legit = 9.265
fedem_ibdist_adv = 10.03

# Local - adv
local_adv_miss = 0.30
local_adv_target = 0.05
local_orig_acc = 0.30
local_ibdist_legit = 50.21
local_ibdist_adv = 87.08

# Fedavg - adv
fedavg_adv_miss = 0.35
fedavg_adv_target =  0.21
fedavg_orig_acc = 0.93
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - adv
fedem_adv_miss = 0.54
fedem_adv_target = 0.073
fedem_orig_acc = 0.71
fedem_ibdist_legit = 14.29
fedem_ibdist_adv = 48.97

In [19]:
# Local Validation Dataset

# Local
local_adv_miss = 0.37
local_adv_target = 0.06
local_orig_acc = 0.99
local_ibdist_legit = 
local_ibdist_adv = 

# Fedavg - Benign
fedavg_adv_miss = 0.00
fedavg_adv_target = 0.85
fedavg_orig_acc = 0.94
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - Benign
fedem_adv_miss = 0.10
fedem_adv_target = 0.44
fedem_orig_acc = 0.94
fedem_ibdist_legit = 
fedem_ibdist_adv = 

# Local - adv
local_adv_miss = 0.29
local_adv_target = 0.05
local_orig_acc = 0.77
local_ibdist_legit = 
local_ibdist_adv = 

# Fedavg - adv
fedavg_adv_miss = 0.33
fedavg_adv_target =  0.24
fedavg_orig_acc = 0.87
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - adv
fedem_adv_miss = 0.49
fedem_adv_target = 0.07
fedem_orig_acc = 0.84
fedem_ibdist_legit = 
fedem_ibdist_adv = 

SyntaxError: invalid syntax (85517529.py, line 7)

In [ ]:
# Local Test (Train) Dataset

# Local
local_adv_miss = 0.38
local_adv_target = 0.06
local_orig_acc = 0.52
local_ibdist_legit = 
local_ibdist_adv = 

# Fedavg - Benign
fedavg_adv_miss = 0.00
fedavg_adv_target = 0.85
fedavg_orig_acc = 0.81
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - Benign
fedem_adv_miss = 0.10
fedem_adv_target = 0.46
fedem_orig_acc = 0.84
fedem_ibdist_legit = 
fedem_ibdist_adv = 

# Local - adv
local_adv_miss = 0.30
local_adv_target = 0.05
local_orig_acc = 0.46
local_ibdist_legit = 
local_ibdist_adv = 

# Fedavg - adv
fedavg_adv_miss = 0.26
fedavg_adv_target = 0.30
fedavg_orig_acc = 0.74
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - adv
fedem_adv_miss = 0.42
fedem_adv_target = 0.09
fedem_orig_acc = 0.74
fedem_ibdist_legit = 
fedem_ibdist_adv = 